In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import re
from ast import literal_eval
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def find_dom_coord(total_x_dist, total_y_dist):
    # odredvanje dominantna koordinata po koi se dvizi strelkata
    
    if total_x_dist > 1.2 * total_y_dist: # znaci deka po x poke se dvizel mausot → x e dominantna koordinata
        return 1 # 1 = x, 2 = y, 0 = nema dom
    elif total_y_dist > 1.2 * total_x_dist:
        return 2
    else:
        return 0

In [404]:
def calculate_density(x, y, num_bins_x=10, num_bins_y=10, num_fields=10):
    # ekranot go deli na 100 polinja i kazva kolkava e gustinata na posetenost na mausot vo sekoe pole
    
    x_bins = np.linspace(0, 1029, num_bins_x + 1)  # Adjust based on your x coordinate range
    y_bins = np.linspace(0, 1017, num_bins_y + 1)  # Adjust based on your y coordinate range

    density_matrix, _, _ = np.histogram2d(x, y, bins=[x_bins, y_bins])

    # Normalize by the total number of movements
    normalized_density = density_matrix / density_matrix.sum()

    my_list = normalized_density.flatten() # Flatten the matrix

    # gi naoga indeksite na num_of_fields polinja vo koi ima najgolema gustina
    indices_of_largest = [index for index, value in sorted(enumerate(my_list), key=lambda x: x[1], reverse=True)[:num_fields]]

    return indices_of_largest

In [405]:
# import plotly.graph_objects as go
# import pandas as pd

# data = [
#     "Filip",
#     [33, 33, 1], [36, 31, 1118], [40, 30, 20], [44, 28, 50], [47, 28, 29], [50, 28, 20], [55, 28, 30], [56, 28, 10],
#     [59, 28, 19], [66, 28, 39], [66, 29, 10], [70, 30, 10], [71, 31, 10], [72, 31, 20], [72, 35, 10], [69, 39, 10],
#     [68, 43, 19], [60, 49, 20], [54, 54, 10], [47, 58, 10], [43, 59, 10], [36, 60, 20], [27, 60, 10], [14, 58, 29],
#     [14, 57, 10], [10, 55, 11], [5, 51, 10], [5, 48, 11], [1, 42, 20], [0, 39, 10], [0, 33, 30], [0, 28, 20], [3, 23, 10],
#     [5, 21, 10], [6, 18, 10], [8, 18, 9], [10, 14, 31]
# ]

# x = [d[0] for d in data[1:]]
# y = [d[1] for d in data[1:]]
# timestamps = [d[2] for d in data[1:]]

# fig = go.Figure(
#     data=[go.Scatter(x=x, y=y, mode='lines+markers', line=dict(color='blue'), name='Movement')],
#     layout=go.Layout(
#         xaxis=dict(range=[min(x), max(x)]),
#         yaxis=dict(range=[min(y), max(y)]),
#         updatemenus=[dict(type='buttons', buttons=[dict(label='Play', method='animate', args=[None, dict(frame=dict(duration=timestamps, redraw=True), fromcurrent=True)])])],
#     ),
#     frames=[go.Frame(data=[go.Scatter(x=x[:k+1], y=y[:k+1], mode='lines+markers', line=dict(color='blue'))]) for k in range(len(x))]
# )

# fig.show()

In [406]:
# Calculate covered area by the mouse movements
def calculate_covered_area(x, y):
    min_x, max_x = min(x), max(x)
    min_y, max_y = min(y), max(y)

    area = (max_x - min_x) * (max_y - min_y)
    return area

In [407]:
def create_density_columns(data, num_fields=3):
    # redniot broj na polinjata vo koi ima najgusto dvizenje na mausot
    
    new_columns = [f'density{i}' for i in range(1, num_fields+1)]
    new_df = pd.DataFrame(data['density'].to_list(), columns=new_columns)
    data = pd.concat([data, new_df], axis=1)    
    data = data.drop('density', axis=1)

    return data

In [408]:
# def calculate_angle_data(angles):
#     if len(angles) < 2:
#         # Handle the case when there are too few movements
#         threshold = 0.0  # Set a default threshold
#     else:
#         threshold = np.percentile(np.abs(np.diff(angles)), 95)
#     direction_changes = np.sum(np.abs(np.diff(angles)) > threshold)
        
#     circular_variance = 1 - np.abs(np.mean(np.exp(1j * angles)))
#     direction_changes = np.sum(np.abs(np.diff(angles)) > threshold)

#     directional_entropy = -np.sum(p * np.log2(p) for p in np.unique(angles, return_counts=True)[1] / len(angles))
    
#     return circular_variance, direction_changes, directional_entropy

In [409]:
# se gleda vo edna sesija najgolem del od vreminjata vo koj opseg bile. vadi golema tocnost na x_val ama mala tocnost na kaggle
def find_list_group(my_list, bound_1, bound_2, bound_3):
    
    # count_1 = count_2 = count_3 = count_4 = 0

    # for value in t:
    #     if value < 10:
    #         count_1 += 1
    #     elif 10 <= value < 12:
    #         count_2 += 1
    #     elif 12 <= value <= 20:
    #         count_3 += 1
    #     elif value > 20:
    #         count_4 += 1

    # counts = [count_1, count_2, count_3, count_4]
    # max_count = max(counts)
    # majority_group = counts.index(max_count) + 1

    count_1 = count_2 = count_3 = count_4 = 0

    for value in my_list:
        if value < bound_1:
            count_1 += 1
        elif bound_1 <= value <= bound_2:
            count_2 += 1
        elif bound_2 <= value <= bound_3:
            count_3 += 1
        else:
            count_4 += 1

    counts = [count_1, count_2, count_3, count_4]
    max_count = max(counts)
    majority_group = counts.index(max_count) + 1

    return majority_group

In [410]:
# nz so e od chatgpt nesto

def box_counting_dimension(x, y):
    # Assume x and y are lists representing mouse coordinates
    x_range = max(x) - min(x)
    y_range = max(y) - min(y)

    # Divide the range into boxes of different sizes
    box_sizes = np.logspace(0.01, 1, num=10)
    counts = []

    for box_size in box_sizes:
        x_bins = np.arange(min(x), max(x) + box_size, box_size)
        y_bins = np.arange(min(y), max(y) + box_size, box_size)

        count = np.sum(np.histogram2d(x, y, bins=[x_bins, y_bins])[0] > 0)
        if count > 0:  # Exclude zero counts
            counts.append(count)

    if len(counts) == 0:
        return 0  # Handle the case where all counts are zero

    # Fit a linear regression to estimate the fractal dimension
    slope, _ = np.polyfit(np.log(1 / box_sizes[:len(counts)]), np.log(counts), 1)

    return -slope  # The fractal dimension is the negative of the slope


In [411]:
# izbroj kolku dvizenja nalevo, nadesno, nagore i nadolu imalo vo edna sesija
# ako istovremeno sme se pridvizile i nagore i nadesno, i count_U i count_R se zgolemvat

def count_movements(direction_list):
    count_L = count_R = count_U = count_D = 0
    for i in range(len(direction_list)):
        # if direction_list[i] == 'L':
        if 'L' in direction_list[i]:
            count_L += 1
        if 'R' in direction_list[i]:
            count_R += 1
        if 'U' in direction_list[i]:
            count_U += 1
        if 'D' in direction_list[i]:
            count_D += 1

    return count_L, count_R, count_U, count_D
        

In [412]:
# odredvanje na nasoka na dvizenje
# ako istovremeno se pridvizime i nagore i nadesno, funkcijata vraka 'RU'

def get_movement_direction(x1, x2, y1, y2):
    x_direction = ''
    y_direction = ''

    # Determine left/right movement
    if x2 < x1:
        x_direction = 'L'
    elif x2 > x1:
        x_direction = 'R'

    # Determine up/down movement
    if y2 > y1:
        y_direction = 'U'
    elif y2 < y1:
        y_direction = 'D'

    # Combine both directions
    if x_direction and y_direction:
        return f'{x_direction}{y_direction}'
    elif x_direction:
        return x_direction
    elif y_direction:
        return y_direction

In [413]:
def count_elements(my_list, bound_1, bound_2, bound_3):
    count_1 = count_2 = count_3 = count_4 = 0
    for element in my_list:
        if element < bound_1:
            count_1 += 1
        elif bound_1 <= element <= bound_2:
            count_2 += 1
        elif bound_2 <= element <= bound_3:
            count_3 += 1
        else:
            count_4 += 1
    return count_1, count_2, count_3, count_4

In [414]:
def create_dataframe(data_name, is_for_testing=False):
    
    # Read data from the text file
    with open(data_name, 'r') as file:
        lines = file.readlines()

    # fig, axs = plt.subplots(400, 3)

    # inicijalizacija
    data = []
    current_label = None
    current_ordinal = None
    x, y, t, x_dist, y_dist, speed, acc, jerk, distance, direction = ([] for _ in range(10))
    dir_changes_x = dir_changes_y = 0 # direction changes
    x_max = y_max = 9
    
    max_movements = 0
    # total_distance = 0
    skip_line = True
    i = 0
    
    for line in lines:

        if line.startswith('-'):
            current_word = line.strip().strip('-')
                
            if current_word[0] in '1234567890': # ova znaci deka zavrsila sesijata i pocnuva nova sesija
                i += 1
                current_ordinal = current_word

                if current_label is not None and current_ordinal is not None: # znaci deka koga vlegvame vo prvata sesija,
                    # nema da se vrsat rabotite podolu

                    dom_coord = find_dom_coord(sum(x_dist), sum(y_dist))
                    area = calculate_covered_area(x, y)
                    density = calculate_density(x, y, num_bins_x=10, num_bins_y=10, num_fields=3)
                    
                    angles = np.arctan2(np.diff(y), np.diff(x))
                    # circular_variance, direction_changes, directional_entropy = calculate_angle_data(angles)
                    
                    num_movements = len(x)
                    if num_movements > max_movements:
                        max_movements = num_movements
                        
                    time = sum(t)

                    # presmetvanje golemina na ekran
                    if(max(x) > x_max):
                        x_max = max(x) 
                    if(max(y) > y_max):
                        y_max = max(y)

                    time_group = find_list_group(t, 10, 12, 20)
                    x_group = find_list_group(x, 100, 300, 500)
                    y_group = find_list_group(y, 100, 300, 500)
                    
                    t1, t2, t3, t4 = count_elements(t, 10, 12, 20)
                    x1, x2, x3, x4 = count_elements(x, 100, 300, 500)
                    y1, y2, y3, y4 = count_elements(y, 100, 300, 500)
                    # fractal_dimension = box_counting_dimension(x, y)
                    count_L, count_R, count_U, count_D = count_movements(direction)

                    # axs[i].plot(x, y)

                    # se preskoknuvaat sesiite kaj sho ima samo 1 dvizenje, osven koga 
                    # gi obrabotvam test podatocite
                    if num_movements > 1 or is_for_testing: 
                        
                        # zakomentirano e oti isprobvam so dodavanje i otstranvanje razlicni odliki
                        
                        
                        
                        # data.append([current_ordinal, current_label, x, y, t, time/num_movements, time, 
                        #              num_movements, total_distance, total_speed/num_movements, max_speed, 
                        #              dir_changes, dir_changes/num_movements, dom_coord, density, area, min(x), 
                        #              max(x), min(y), max(y), np.mean(x), np.mean(y)])

                        # vo slucaj koga listite se prazni (slucaj vo mouse_prediction podatocite)
                        if len(speed) == 0: 
                            speed = [0]
                        if len(acc) == 0: 
                            acc = [0]
                        if len(jerk) == 0: 
                            jerk = [0]
                        if len(distance) == 0: 
                            distance = [0]
                        if len(x_dist) == 0: 
                            x_dist = [0]
                        if len(y_dist) == 0: 
                            y_dist = [0]
                        if len(t) == 0: 
                            t = [0]
                        if len(angles) == 0: 
                            angles = [0]
                            
                    
                        data.append([direction, current_ordinal, current_label, x, y, t,
                                     # time_group, x_group, y_group,
                                     x1/num_movements, x2/num_movements, x3/num_movements, x4/num_movements, 
                                     y1/num_movements, y2/num_movements, y3/num_movements, y4/num_movements, 
                                     t1/num_movements, t2/num_movements, t3/num_movements, t4/num_movements,
                                     # sum(t), min(t), max(t), np.mean(t), np.std(t), sum(x), min(x), max(x), np.mean(x), np.std(x), sum(y), min(y), max(y), np.mean(y), np.std(y),
                                     min(t), max(t), sum(t)/num_movements, max(t)-min(t), np.std(t), np.median(t), np.var(t),
                                     max(x), np.mean(x), np.std(x), np.median(x), np.var(x),
                                     max(y), np.mean(y), np.std(y), np.median(y), np.var(y),
                                     
                                     
                                     # sum(distance), min(distance), max(distance), np.mean(distance), np.std(distance),
                                     # sum(x_dist), min(x_dist), max(x_dist), np.mean(x_dist), np.std(x_dist),
                                     # sum(y_dist), min(y_dist), max(y_dist), np.mean(y_dist), np.std(y_dist),
                                                                         
                                     min(distance), max(distance), np.mean(distance), np.std(distance), max(distance)-min(distance), np.median(distance), np.var(distance),
                                     min(x_dist), max(x_dist), np.mean(x_dist), np.std(x_dist), max(x_dist)-min(x_dist), np.median(x_dist), np.var(x_dist),
                                     min(y_dist), max(y_dist), np.mean(y_dist), np.std(y_dist), max(y_dist)-min(y_dist), np.median(y_dist), np.var(y_dist),
 
                                     max(speed), np.mean(speed), np.std(speed), np.median(speed), np.var(speed),
                                     # max(acc), np.mean(acc), np.std(acc), np.median(acc), np.var(acc),
                                     
                                     # min(speed), max(speed), np.mean(speed), np.median(speed), np.var(speed), np.max(speed) - np.min(speed), np.std(speed),
                                     # min(acc), max(acc), np.mean(acc), np.median(acc), np.var(acc), np.max(acc) - np.min(acc), np.std(acc), np.median(acc), np.var(acc),
                                     # min(jerk), max(jerk), np.mean(jerk), np.median(jerk), np.var(jerk), np.max(jerk) - np.min(jerk), np.std(jerk),
                                     
                                     # min(angles), max(angles), np.mean(angles), np.median(angles), np.var(angles), np.max(angles) - np.min(angles), np.std(angles),
                                     # min(angles), max(angles), np.mean(angles), np.max(angles) - np.min(angles), np.std(angles),
                                     # circular_variance, direction_changes, directional_entropy,
                        
                                     num_movements, dir_changes_x/num_movements, dir_changes_y/num_movements,
                                     count_L/num_movements, count_R/num_movements, count_U/num_movements, count_D/num_movements,
                                     # dom_coord, density, area
                                    ])


                    # reset na promenlivite
                    x, y, t, x_dist, y_dist, speed, acc, jerk, distance, direction = ([] for _ in range(10))
                    dir_changes_x = dir_changes_y = 0 # direction changes
                    total_x_dist = 0
                    total_y_dist = 0
                    total_distance = 0
                    skip_line = True
                
            else: # ako stringot posle "-" ne e broj
                current_label = current_word            
    
        if line.startswith('[') and line.endswith(']\n'):

            # prvite vrednosti ne gi zemam vo predvid
            # if skip_line == False:
            
            coordinates = literal_eval(line) # stringot go pretvora vo lista so broevi

            # se preskoknuvat 2 posledovatelni dvizenja sho imat ista x i y koordinata
            if len(x) == 0 or (coordinates[0] != x[-1] or coordinates[1] != y[-1]):
                x.append(coordinates[0])
                y.append(coordinates[1])
                if skip_line == False:
                    t.append(coordinates[2])

                # presmetvanje brzina i rastojanie
                if len(x) >= 2:
                    direction.append(get_movement_direction(x[-2], x[-1], y[-2], y[-1]))
                    
                    distance.append(np.sqrt(((x[-1]-x[-2])**2) + ((y[-1]-y[-2])**2)))
                    x_dist.append(abs(x[-1]-x[-2]))
                    y_dist.append(abs(y[-1]-y[-2]))
                    # distance.append(np.sqrt(x_dist[-1]**2 + y_dist[-1]**2))
                    speed.append(distance[-1] / t[-1])
                    acc.append(speed[-1] / t[-1])
                    jerk.append(acc[-1] / t[-1])

                # broenje kolku pati se smenala nasokata na dvizenje na mausot
                if len(x) >= 3:
                    if not ((x[-1] > x[-2] > x[-3]) or (x[-1] < x[-2] < x[-3])):
                        dir_changes_x += 1
                    if not((y[-1] < y[-2] < y[-3]) or (y[-1] > y[-2] > y[-3])):
                        dir_changes_y += 1

            skip_line = False
            

    # df = pd.DataFrame(data, columns=['ordinal_number', 'label', 'x', 'y', 't', 'avg_time', 'time', 'num_movements', 
    #                                  'total_distance', 'avg_speed', 'max_speed', 'direction_changes', 
    #                                  'avg dir changes per movements', 'dom coord', 'density', 'area', 'min_x', 'max_x', 
    #                                  'min_y', 'max_y', 'mean_x', 'mean_y'])
    
    # mean(distance) davase greski golemi!
    df = pd.DataFrame(data, columns=[
                'direction', 'current_ordinal', 'label', 'x', 'y', 't',
                # 'time_group', 'x_group', 'y_group',
                'x1', 'x2', 'x3', 'x4', 
                'y1', 'y2', 'y3', 'y4', 
                't1', 't2', 't3', 't4',
                # 'sum(t)', 'min(t)', 'max(t)', 'np.mean(t)', 'np.std(t)', 'sum(x)', 'min(x)', 'max(x)', 'np.mean(x)', 'np.std(x)', 'sum(y)', 'min(y)', 'max(y)', 'np.mean(y)', 'np.std(y)',
                'min(t)', 'max(t)', 'np.mean(t)', 'max(t)-min(t)', 'np.std(t)', 'np.median(t)', 'np.var(t)',
                'max(x)', 'np.mean(x)', 'np.std(x)', 'np.median(x)', 'np.var(x)',
                'max(y)', 'np.mean(y)', 'np.std(y)', 'np.median(y)', 'np.var(y)',
        
                'min(distance)', 'max(distance)', 'np.mean(distance)', 'np.std(distance)', 'max(distance)-min(distance)', 'np.median(distance)', 'np.var(distance)',
                'min(x_dist)', 'max(x_dist)', 'np.mean(x_dist)', 'np.std(x_dist)', 'max(x_dist)-min(x_dist)', 'np.median(x_dist)', 'np.var(x_dist)',
                'min(y_dist)', 'max(y_dist)', 'np.mean(y_dist)', 'np.std(y_dist)', 'max(y_dist)-min(y_dist)', 'np.median(y_dist)', 'np.var(y_dist)',

                # 'sum(distance)', 'min(distance)', 'max(distance)', 'np.std(distance)',
                # 'sum(x_dist)', 'min(x_dist)', 'max(x_dist)', 'np.std(x_dist)',
                # 'sum(y_dist)', 'min(y_dist)', 'max(y_dist)', 'np.std(y_dist)',
        
                # 'min(speed)', 'max(speed)', 'np.mean(speed)', 'np.median(speed)', 'np.var(speed)', 'np.max(speed) - np.min(speed)', 'np.std(speed)',
                # 'min(acc)', 'max(acc)', 'np.mean(acc)', 'np.median(acc)', 'np.var(acc)', 'np.max(acc) - np.min(acc)', 'np.std(acc)',

                'max(speed)', 'np.mean(speed)', 'np.std(speed)', 'np.median(speed)', 'np.var(speed)',
                # 'max(acc)', 'np.mean(acc)', 'np.std(acc)', 'np.median(acc)', 'np.var(acc)',
                # 'min(jerk)', 'max(jerk)', 'np.mean(jerk)', 'np.median(jerk)', 'np.var(jerk)', 'np.max(jerk) - np.min(jerk)', 'np.std(jerk)',
        
                # 'min(angles)', 'max(angles)', 'np.mean(angles)', 'np.median(angles)', 'np.var(angles)', 'np.max(angles) - np.min(angles)', 'np.std(angles)',
                # 'min(angles)', 'max(angles)', 'np.mean(angles)', 'np.max(angles) - np.min(angles)', 'np.std(angles)',
                # 'circular_variance', 'direction_changes', 'directional_entropy',
        
                'num_movements', 'dir_changes_x/num_movements', 'dir_changes_y/num_movements',
                'count_L', 'count_R', 'count_U', 'count_D',
                # 'dom_coord', 'density', 'area'
            ])

    df = df.drop('current_ordinal', axis=1)

    # ova ne mi treba vo momentot ama dodava nuli vo listite koi imat pomalku elementi od max_movements
    # df['x'] = df['x'].apply(lambda x: x + [0] * (max_movements - len(x)))
    # df['y'] = df['y'].apply(lambda x: x + [0] * (max_movements - len(x)))
    # df['t'] = df['t'].apply(lambda x: x + [0] * (max_movements - len(x)))

    # eksperiment
    df = df.drop('x', axis=1)
    df = df.drop('y', axis=1)
    df = df.drop('t', axis=1)
    df = df.drop('direction', axis=1)

    # df = create_density_columns(df, num_fields=3)

    return df, x_max, y_max

In [415]:
train_data, x_max, y_max = create_dataframe('mouse_data.txt')

In [416]:
train_data

,label,x1,x2,x3,x4,y1,y2,y3,y4,t1,...,np.std(speed),np.median(speed),np.var(speed),num_movements,dir_changes_x/num_movements,dir_changes_y/num_movements,count_L,count_R,count_U,count_D
0,Dushko,0.012195,0.000000,0.097561,0.890244,0.097561,0.085366,0.780488,0.036585,0.012195,...,1.532884,0.217571,2.349733,82,0.402439,0.256098,0.402439,0.378049,0.621951,0.195122
1,Dushko,1.000000,0.000000,0.000000,0.000000,0.850000,0.150000,0.000000,0.000000,0.000000,...,0.434554,0.212459,0.188838,20,0.550000,0.050000,0.300000,0.250000,0.450000,0.500000
2,Dushko,1.000000,0.000000,0.000000,0.000000,0.840000,0.160000,0.000000,0.000000,0.040000,...,0.232427,0.240625,0.054022,25,0.480000,0.000000,0.640000,0.000000,0.960000,0.000000
3,Dushko,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.486057,0.098640,0.236252,7,0.571429,0.000000,0.000000,0.285714,0.000000,0.857143
4,Dushko,0.735294,0.264706,0.000000,0.000000,0.294118,0.705882,0.000000,0.000000,0.029412,...,1.217683,0.263523,1.482753,34,0.411765,0.205882,0.647059,0.058824,0.411765,0.382353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1074,Vesna,0.476190,0.523810,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.357143,...,0.937847,0.250000,0.879558,42,0.261905,0.476190,0.547619,0.309524,0.214286,0.452381
1075,Vesna,0.698113,0.301887,0.000000,0.000000,0.462264,0.537736,0.000000,0.000000,0.481132,...,0.760718,0.395285,0.578692,106,0.179245,0.358491,0.462264,0.452830,0.547170,0.198113
1076,Vesna,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.312500,...,0.623248,0.395285,0.388437,16,0.375000,0.187500,0.375000,0.312500,0.375000,0.437500
1077,Vesna,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.384615,...,0.794240,0.508775,0.630817,13,0.538462,0.153846,0.384615,0.230769,0.384615,0.538462


In [417]:
train_data.describe()

,x1,x2,x3,x4,y1,y2,y3,y4,t1,t2,...,np.std(speed),np.median(speed),np.var(speed),num_movements,dir_changes_x/num_movements,dir_changes_y/num_movements,count_L,count_R,count_U,count_D
count,1079.000000,1079.000000,1079.000000,1079.000000,1079.000000,1079.000000,1079.000000,1079.000000,1079.000000,1079.000000,...,1079.000000,1079.000000,1079.000000,1079.000000,1079.000000,1079.000000,1079.000000,1079.000000,1079.000000,1079.000000
mean,0.694809,0.245005,0.042847,0.017339,0.687631,0.254767,0.049907,0.007694,0.143732,0.234323,...,2.253263,1.222955,109.584363,47.422614,0.308767,0.209214,0.381638,0.342299,0.383514,0.411732
std,0.319960,0.271098,0.118341,0.094033,0.319987,0.275172,0.147108,0.054830,0.157889,0.152652,...,10.227615,13.826165,2362.970634,52.726561,0.205553,0.181295,0.274814,0.268348,0.285481,0.296990
min,0.011111,0.000000,0.000000,0.000000,0.006289,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000091,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.411765,0.000000,0.000000,0.000000,0.412404,0.000000,0.000000,0.000000,0.000000,0.132704,...,0.387391,0.200000,0.150072,16.000000,0.179366,0.066667,0.148171,0.105263,0.067816,0.188988
50%,0.792453,0.152778,0.000000,0.000000,0.767241,0.175439,0.000000,0.000000,0.100629,0.239437,...,0.777161,0.316228,0.603979,31.000000,0.282609,0.184615,0.391304,0.333333,0.411765,0.382353
75%,1.000000,0.474937,0.000000,0.000000,1.000000,0.480824,0.000000,0.000000,0.223217,0.344006,...,1.756608,0.485158,3.085674,60.000000,0.402439,0.305159,0.564435,0.500000,0.588638,0.614510
max,1.000000,0.971429,0.809524,0.977778,1.000000,0.977778,0.960000,0.758065,0.714286,0.709677,...,274.354227,311.443414,75270.242008,537.000000,0.928571,0.945946,0.972222,0.955556,0.971429,0.964286


In [418]:
# import matplotlib.pyplot as plt
# import numpy as np
# fig, axs = plt.subplots(5, int(len(train_data)/5), figsize=(15, 10), sharex=True, sharey=True)
# for i in range(len(train_data)):
#     xplot = train_data['x'].iloc[i]
#     # print(xplot)
#     yplot = train_data['y'].iloc[i]
#     axs[i].plot(xplot, yplot)
#     axs[i].set_title(train_data['label'.iloc[i]])
    

In [419]:
# enkodiranje na dadenite coloni: num_bins = broj na grupi vo koi ke gi oddelime podatocite od sekoja kolona. vo sekoja grupa se praj da ima ednakov broj na podatocite
# pa taka na primer ako imame podatoci [1 2 3 4 50 100 150 200] i num_bins=2, togas 50% od podatocite ke se grupa 1 (podatocite 1 2 3 4) i 50% ke se grupa 2 (50 100 150 200).
# dvete grupi imaat razlicna golemina na opsegot (prvata e od 1 do 4, vtorata e od 50 do 200) ama dvete vo niv sodrzat ednakov broj podatoci

def encode_data(df, num_bins=20):
    columns=[
                'min(t)', 'max(t)', 'np.mean(t)', 'max(t)-min(t)', 'np.std(t)', 'np.median(t)', 'np.var(t)',
                'max(x)', 'np.mean(x)', 'np.std(x)', 'np.median(x)', 'np.var(x)',
                'max(y)', 'np.mean(y)', 'np.std(y)', 'np.median(y)', 'np.var(y)',
        
                'min(distance)', 'max(distance)', 'np.mean(distance)', 'np.std(distance)', 'max(distance)-min(distance)', 'np.median(distance)', 'np.var(distance)',
                'min(x_dist)', 'max(x_dist)', 'np.mean(x_dist)', 'np.std(x_dist)', 'max(x_dist)-min(x_dist)', 'np.median(x_dist)', 'np.var(x_dist)',
                'min(y_dist)', 'max(y_dist)', 'np.mean(y_dist)', 'np.std(y_dist)', 'max(y_dist)-min(y_dist)', 'np.median(y_dist)', 'np.var(y_dist)',
                'max(speed)', 'np.mean(speed)', 'np.std(speed)', 'np.median(speed)', 'np.var(speed)',
            ]
    
    for column in columns:
    # Use qcut to create bins based on percentiles
        df[column] = pd.qcut(df[column], q=num_bins, labels=False, precision=1, duplicates='drop') + 1

    return df

In [420]:
# train_data = encode_data(train_data)
# train_data

In [421]:
def create_x_y(data):
    X_train = data.drop('label', axis=1)
    y_train = data['label']

    return X_train, y_train

In [422]:
X_train, y_train = create_x_y(train_data)
X_train

,x1,x2,x3,x4,y1,y2,y3,y4,t1,t2,...,np.std(speed),np.median(speed),np.var(speed),num_movements,dir_changes_x/num_movements,dir_changes_y/num_movements,count_L,count_R,count_U,count_D
0,0.012195,0.000000,0.097561,0.890244,0.097561,0.085366,0.780488,0.036585,0.012195,0.243902,...,1.532884,0.217571,2.349733,82,0.402439,0.256098,0.402439,0.378049,0.621951,0.195122
1,1.000000,0.000000,0.000000,0.000000,0.850000,0.150000,0.000000,0.000000,0.000000,0.050000,...,0.434554,0.212459,0.188838,20,0.550000,0.050000,0.300000,0.250000,0.450000,0.500000
2,1.000000,0.000000,0.000000,0.000000,0.840000,0.160000,0.000000,0.000000,0.040000,0.080000,...,0.232427,0.240625,0.054022,25,0.480000,0.000000,0.640000,0.000000,0.960000,0.000000
3,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.486057,0.098640,0.236252,7,0.571429,0.000000,0.000000,0.285714,0.000000,0.857143
4,0.735294,0.264706,0.000000,0.000000,0.294118,0.705882,0.000000,0.000000,0.029412,0.205882,...,1.217683,0.263523,1.482753,34,0.411765,0.205882,0.647059,0.058824,0.411765,0.382353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1074,0.476190,0.523810,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.357143,0.000000,...,0.937847,0.250000,0.879558,42,0.261905,0.476190,0.547619,0.309524,0.214286,0.452381
1075,0.698113,0.301887,0.000000,0.000000,0.462264,0.537736,0.000000,0.000000,0.481132,0.000000,...,0.760718,0.395285,0.578692,106,0.179245,0.358491,0.462264,0.452830,0.547170,0.198113
1076,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.312500,0.000000,...,0.623248,0.395285,0.388437,16,0.375000,0.187500,0.375000,0.312500,0.375000,0.437500
1077,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.384615,0.000000,...,0.794240,0.508775,0.630817,13,0.538462,0.153846,0.384615,0.230769,0.384615,0.538462


In [423]:
y_train

0       Dushko
1       Dushko
2       Dushko
3       Dushko
4       Dushko
         ...  
1074     Vesna
1075     Vesna
1076     Vesna
1077     Vesna
1078     Vesna
Name: label, Length: 1079, dtype: object

In [424]:
# treniranje na celoto mnozestvo
X_combined = X_train
y_combined = y_train

# podelba na mnozestvoto na train i val
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=22)

In [425]:
X_train

,x1,x2,x3,x4,y1,y2,y3,y4,t1,t2,...,np.std(speed),np.median(speed),np.var(speed),num_movements,dir_changes_x/num_movements,dir_changes_y/num_movements,count_L,count_R,count_U,count_D
889,0.207792,0.298701,0.493506,0.0,0.298701,0.701299,0.0,0.0,0.090909,0.376623,...,0.594999,0.368083,0.354024,77,0.142857,0.506494,0.584416,0.298701,0.493506,0.116883
969,0.372727,0.627273,0.000000,0.0,1.000000,0.000000,0.0,0.0,0.063636,0.336364,...,0.613699,0.186339,0.376626,110,0.363636,0.336364,0.309091,0.463636,0.400000,0.345455
736,1.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.0,0.0,0.000000,0.111111,...,0.134856,0.190312,0.018186,9,0.777778,0.000000,0.000000,0.000000,0.000000,0.888889
996,1.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.0,0.0,0.100000,0.100000,...,0.069778,0.100000,0.004869,10,0.000000,0.700000,0.900000,0.000000,0.300000,0.000000
611,0.800000,0.200000,0.000000,0.0,1.000000,0.000000,0.0,0.0,0.000000,0.160000,...,0.441274,0.160256,0.194722,25,0.280000,0.360000,0.520000,0.280000,0.000000,0.680000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,1.000000,0.000000,0.000000,0.0,0.675676,0.324324,0.0,0.0,0.567568,0.000000,...,1.701829,0.708184,2.896223,37,0.351351,0.027027,0.432432,0.297297,0.405405,0.567568
960,0.145833,0.854167,0.000000,0.0,0.125000,0.875000,0.0,0.0,0.104167,0.354167,...,1.319421,0.117688,1.740873,48,0.333333,0.229167,0.541667,0.250000,0.750000,0.104167
812,0.495327,0.504673,0.000000,0.0,0.411215,0.588785,0.0,0.0,0.168224,0.383178,...,2.068403,0.370672,4.278290,107,0.177570,0.336449,0.551402,0.364486,0.691589,0.056075
132,0.470588,0.529412,0.000000,0.0,1.000000,0.000000,0.0,0.0,0.000000,0.058824,...,0.436769,0.235007,0.190767,17,0.058824,0.235294,0.000000,0.882353,0.000000,0.823529


In [426]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 863 entries, 889 to 885
Data columns (total 62 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   x1                           863 non-null    float64
 1   x2                           863 non-null    float64
 2   x3                           863 non-null    float64
 3   x4                           863 non-null    float64
 4   y1                           863 non-null    float64
 5   y2                           863 non-null    float64
 6   y3                           863 non-null    float64
 7   y4                           863 non-null    float64
 8   t1                           863 non-null    float64
 9   t2                           863 non-null    float64
 10  t3                           863 non-null    float64
 11  t4                           863 non-null    float64
 12  min(t)                       863 non-null    int64  
 13  max(t)                 

In [427]:
# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

scaler_combined = StandardScaler()
X_combined = scaler_combined.fit_transform(X_combined)

## SVM model

In [368]:
from sklearn.model_selection import cross_val_score

In [369]:
# svm_model = SVC(kernel='poly', degree=10)  # You can experiment with different kernels
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)
y_val_pred_rf = svm_model.predict(X_val)
accuracy_rf = accuracy_score(y_val, y_val_pred_rf)
print(f'SVM Validation Accuracy: {accuracy_rf * 100:.2f}%')

scores = cross_val_score(svm_model, X_train, y_train, cv=5, scoring='accuracy')
print(f'Cross-Validation Accuracy: {np.mean(scores) * 100:.2f}%')

SVM Validation Accuracy: 84.72%
Cross-Validation Accuracy: 80.99%


## Random forest model

In [428]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

# Random forest
# Create and train the Random Forest model
# rf_model = RandomForestClassifier(n_estimators=200, random_state=42, max_depth=30, min_samples_split=5, min_samples_leaf=2)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate the model on the validation set
y_val_pred_rf = rf_model.predict(X_val)
accuracy_rf = accuracy_score(y_val, y_val_pred_rf)
print(f'Random Forest Validation Accuracy: {accuracy_rf * 100:.2f}%')

Random Forest Validation Accuracy: 89.35%


In [371]:
scores = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='accuracy')
print(f'Cross-Validation Accuracy: {np.mean(scores) * 100:.2f}%')

Cross-Validation Accuracy: 87.60%


In [372]:
rf_model_combined = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_combined.fit(X_combined, y_combined)

RandomForestClassifier(random_state=42)

## Nevronski mrezi model

In [177]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l1_l2
from keras.layers.core import Dense, Dropout
from keras.layers import BatchNormalization

In [178]:
# pretvoranje na y_train i y_val od string vo int
from sklearn.preprocessing import LabelEncoder

label_encoder = label_encoder_combined = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_combined_encoded = label_encoder_combined.fit_transform(y_combined)

In [189]:
num_features = X_train.shape[1]

In [190]:
def create_nn_model():
    nn_model = keras.Sequential([
        layers.InputLayer(input_shape=(num_features,)),
        
        layers.Dense(128, activation='tanh'),
        layers.Dropout(0.1),
        BatchNormalization(),
        # layers.Dense(128, activation='relu'),
        # layers.Dropout(0.2), 
        layers.Dense(128, activation='tanh'),
        layers.Dropout(0.),
        BatchNormalization(),
        layers.Dense(128, activation='tanh'),
        layers.Dropout(0.1), 
        layers.Dense(128, activation='tanh'),
        layers.Dropout(0.1),
        layers.Dense(32, activation='tanh'),       
        layers.Dense(4, activation='softmax') # Output layer with 4 units (for 4 classes) and softmax activation
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    return nn_model

In [191]:
# num_features = X_train.shape[1]

# nn_model = nn_model_combined = keras.Sequential([
#     layers.InputLayer(input_shape=(num_features,)),
    
#     layers.Dense(128, activation='tanh'),
#     layers.Dropout(0.1),
#     BatchNormalization(),
#     # layers.Dense(128, activation='relu'),
#     # layers.Dropout(0.2), 
#     layers.Dense(128, activation='tanh'),
#     layers.Dropout(0.),
#     BatchNormalization(),
#     layers.Dense(128, activation='tanh'),
#     layers.Dropout(0.1), 
#     layers.Dense(128, activation='tanh'),
#     layers.Dropout(0.1),
#     layers.Dense(32, activation='tanh'),       
#     layers.Dense(4, activation='softmax') # Output layer with 4 units (for 4 classes) and softmax activation
# ])

# optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [180]:
# num_features = X_train.shape[1]

# nn_model = keras.Sequential(
#     # [keras.layers.Dense(300, activation='relu', input_shape=(num_features,), kernel_regularizer=l1_l2(l1=0.5, l2=0.7)), # kernel_regularizer=l1(0.1) otstrani, mozda to praj underfitting
#     # #keras.layers.Dense(200, activation='relu', input_shape=(num_features,)),
#     # Dropout(0.05),
#     # BatchNormalization(),
#     # #BatchNormalization(),
#     # keras.layers.Dense(200, activation='relu', kernel_regularizer=l1_l2(l1=0.5, l2=0.7)),
#     # Dropout(0.05),
#     # BatchNormalization(),
#     # keras.layers.Dense(200, activation='relu'),
    
#     # keras.layers.Dense(200, activation='relu'),
#     # BatchNormalization(),
#     # #Dropout(0.4),

#     [keras.layers.Dense(100, activation='relu', kernel_regularizer=l1_l2(l1=0.5, l2=0.7)),
#     Dropout(0.02),
#     keras.layers.Dense(100, activation='relu', kernel_regularizer=l1_l2(l1=0.5, l2=0.7)),
#     Dropout(0.02),
#     #BatchNormalization(),
#     #BatchNormalization(),
#     keras.layers.Dense(50, activation='relu', kernel_regularizer=l1_l2(l1=0.5, l2=0.7)),
#     #BatchNormalization(),
#     #BatchNormalization(),
#     keras.layers.Dense(10, activation='relu', kernel_regularizer=l1_l2(l1=0.5, l2=0.7)),
#     #keras.layers.Dense(5, activation='relu'),
#     #BatchNormalization(),
#     #Dropout(0.05),
#     #keras.layers.Dense(10, activation='relu'),
#     #keras.layers.Dense(10, activation='relu'),
#     keras.layers.Dense(4, activation='softmax')  # Output layer for classification
# ])
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [192]:
nn_model = create_nn_model()
nn_model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',  # Use 'categorical_crossentropy' for one-hot encoded labels
              metrics=['accuracy'])

# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Assuming X_val and y_val_encoded are your validation data
history = nn_model.fit(X_train, y_train_encoded, epochs=150, 
                       validation_data=(X_val, y_val_encoded),
                       callbacks=[early_stopping])

Epoch 1/150
27/27 [==============================] - 1s 7ms/step - loss: 0.8915 - accuracy: 0.6385 - val_loss: 0.5245 - val_accuracy: 0.7824
Epoch 2/150
27/27 [==============================] - 0s 3ms/step - loss: 0.5766 - accuracy: 0.7868 - val_loss: 0.4947 - val_accuracy: 0.8009
Epoch 3/150
27/27 [==============================] - 0s 3ms/step - loss: 0.5009 - accuracy: 0.8204 - val_loss: 0.5005 - val_accuracy: 0.8148
Epoch 4/150
27/27 [==============================] - 0s 3ms/step - loss: 0.4767 - accuracy: 0.8181 - val_loss: 0.4927 - val_accuracy: 0.7963
Epoch 5/150
27/27 [==============================] - 0s 3ms/step - loss: 0.3954 - accuracy: 0.8470 - val_loss: 0.4661 - val_accuracy: 0.8056
Epoch 6/150
27/27 [==============================] - 0s 3ms/step - loss: 0.3953 - accuracy: 0.8470 - val_loss: 0.4267 - val_accuracy: 0.8380
Epoch 7/150
27/27 [==============================] - 0s 3ms/step - loss: 0.3908 - accuracy: 0.8331 - val_loss: 0.4275 - val_accuracy: 0.8287
Epoch 8/150
2

In [48]:
# nn_model_combined.compile(optimizer=optimizer,
#               loss='sparse_categorical_crossentropy',  # Use 'categorical_crossentropy' for one-hot encoded labels
#               metrics=['accuracy'])

# # Define the EarlyStopping callback
# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# # Assuming X_val and y_val_encoded are your validation data
# history = nn_model.fit(X_combined, y_combined_encoded, epochs=150)

## Kombinacija od modeli

In [195]:
# # NN
# from keras.wrappers.scikit_learn import KerasClassifier
# nn_model_classifier = KerasClassifier(build_fn=create_nn_model, epochs=100, batch_size=64, verbose=1)
# nn_model = nn_model_classifier
# nn_model.fit(X_train, y_train, )

In [373]:
from sklearn.tree import DecisionTreeClassifier
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)

DecisionTreeClassifier()

In [374]:
from sklearn.ensemble import VotingClassifier

# Get predictions from individual models
pred1 = rf_model.predict(X_val)
pred2 = dt_model.predict(X_val)
pred3 = svm_model.predict(X_val)
# pred4 = nn_model.predict(X_val)

# pred4_classes = np.argmax(pred4, axis=1)
# # vrati gi enkodiranite vrednosti vo stringovi
# pred4_classes = label_encoder.inverse_transform(pred4_classes)

# Create a voting ensemble
voting_classifier = VotingClassifier(estimators=[('model1', rf_model), ('model2', dt_model), ('model3', svm_model)], voting='hard')

# Train the voting ensemble on the training data
voting_classifier.fit(X_train, y_train)

# Get predictions from the voting ensemble
ensemble_pred = voting_classifier.predict(X_val)

# Evaluate individual models
accuracy_model1 = accuracy_score(y_val, pred1)
accuracy_model2 = accuracy_score(y_val, pred2)
accuracy_model3 = accuracy_score(y_val, pred3)
# accuracy_model4 = accuracy_score(y_val, pred4_classes)

# Evaluate the voting ensemble
accuracy_ensemble = accuracy_score(y_val, ensemble_pred)

print(f"Accuracy of rf_model: {accuracy_model1:.3f}")
print(f"Accuracy of dt_model: {accuracy_model2:.3f}")
print(f"Accuracy of svm_model: {accuracy_model3:.3f}")
# print(f"Accuracy of Model 4: {accuracy_model4:.2f}")
print(f"Accuracy of Voting Ensemble: {accuracy_ensemble:.3f}")

Accuracy of rf_model: 0.894
Accuracy of dt_model: 0.884
Accuracy of svm_model: 0.847
Accuracy of Voting Ensemble: 0.894


# Citanje test data

In [429]:
test_data, _, _ = create_dataframe('mouse_prediction.txt', is_for_testing=True)

In [430]:
test_data

,label,x1,x2,x3,x4,y1,y2,y3,y4,t1,...,np.std(speed),np.median(speed),np.var(speed),num_movements,dir_changes_x/num_movements,dir_changes_y/num_movements,count_L,count_R,count_U,count_D
0,?,1.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.000000,0.0,0.000000,...,0.000000,0.021520,0.000000,2,0.000000,0.000000,0.000000,0.500000,0.000000,0.500000
1,?,1.000000,0.000000,0.000000,0.000000,0.092593,0.907407,0.000000,0.0,0.462963,...,0.978310,0.291667,0.957091,54,0.407407,0.240741,0.314815,0.481481,0.481481,0.351852
2,?,0.183333,0.750000,0.066667,0.000000,0.050000,0.400000,0.550000,0.0,0.583333,...,0.939679,0.400737,0.882996,180,0.238889,0.188889,0.311111,0.566667,0.588889,0.311111
3,?,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.151515,...,0.409787,0.190809,0.167925,33,0.363636,0.212121,0.212121,0.545455,0.272727,0.636364
4,?,0.675000,0.325000,0.000000,0.000000,0.375000,0.625000,0.000000,0.0,0.225000,...,0.941609,0.555556,0.886628,40,0.150000,0.275000,0.500000,0.375000,0.300000,0.550000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,?,0.013699,0.671233,0.287671,0.027397,0.068493,0.726027,0.205479,0.0,0.547945,...,6.843607,1.344112,46.834963,73,0.301370,0.178082,0.479452,0.356164,0.520548,0.424658
129,?,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.500000,...,0.000000,11.704700,0.000000,2,0.000000,0.000000,0.500000,0.000000,0.500000,0.000000
130,?,0.500000,0.500000,0.000000,0.000000,0.500000,0.500000,0.000000,0.0,0.000000,...,0.000000,0.187200,0.000000,2,0.000000,0.000000,0.000000,0.500000,0.000000,0.500000
131,?,1.000000,0.000000,0.000000,0.000000,0.800000,0.200000,0.000000,0.0,0.100000,...,0.383071,0.222222,0.146744,20,0.400000,0.150000,0.750000,0.000000,0.850000,0.000000


In [431]:
X_test, y_test = create_x_y(test_data)
X_test

,x1,x2,x3,x4,y1,y2,y3,y4,t1,t2,...,np.std(speed),np.median(speed),np.var(speed),num_movements,dir_changes_x/num_movements,dir_changes_y/num_movements,count_L,count_R,count_U,count_D
0,1.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.021520,0.000000,2,0.000000,0.000000,0.000000,0.500000,0.000000,0.500000
1,1.000000,0.000000,0.000000,0.000000,0.092593,0.907407,0.000000,0.0,0.462963,0.000000,...,0.978310,0.291667,0.957091,54,0.407407,0.240741,0.314815,0.481481,0.481481,0.351852
2,0.183333,0.750000,0.066667,0.000000,0.050000,0.400000,0.550000,0.0,0.583333,0.000000,...,0.939679,0.400737,0.882996,180,0.238889,0.188889,0.311111,0.566667,0.588889,0.311111
3,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.151515,0.121212,...,0.409787,0.190809,0.167925,33,0.363636,0.212121,0.212121,0.545455,0.272727,0.636364
4,0.675000,0.325000,0.000000,0.000000,0.375000,0.625000,0.000000,0.0,0.225000,0.225000,...,0.941609,0.555556,0.886628,40,0.150000,0.275000,0.500000,0.375000,0.300000,0.550000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,0.013699,0.671233,0.287671,0.027397,0.068493,0.726027,0.205479,0.0,0.547945,0.000000,...,6.843607,1.344112,46.834963,73,0.301370,0.178082,0.479452,0.356164,0.520548,0.424658
129,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.500000,0.000000,...,0.000000,11.704700,0.000000,2,0.000000,0.000000,0.500000,0.000000,0.500000,0.000000
130,0.500000,0.500000,0.000000,0.000000,0.500000,0.500000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.187200,0.000000,2,0.000000,0.000000,0.000000,0.500000,0.000000,0.500000
131,1.000000,0.000000,0.000000,0.000000,0.800000,0.200000,0.000000,0.0,0.100000,0.200000,...,0.383071,0.222222,0.146744,20,0.400000,0.150000,0.750000,0.000000,0.850000,0.000000


In [432]:
# X_test = encode_data(X_test)
# X_test

In [433]:
X_test = scaler.transform(X_test)
# X_test = scaler_combined.transform(X_test)

### predvidvanje so nn_model

In [54]:
y_test_pred = nn_model.predict(X_test)
y_pred_classes = np.argmax(y_test_pred, axis=1)
# vrati gi enkodiranite vrednosti vo stringovi
y_pred_classes = label_encoder.inverse_transform(y_pred_classes)
y_pred_classes

array(['Vesna', 'Filip', 'Vesna', 'Filip', 'Dushko', 'Dushko', 'Vesna',
       'Dushko', 'Dushko', 'Vesna', 'Filip', 'Dushko', 'Filip', 'Stefan',
       'Dushko', 'Filip', 'Dushko', 'Dushko', 'Dushko', 'Filip', 'Dushko',
       'Dushko', 'Stefan', 'Dushko', 'Filip', 'Vesna', 'Dushko', 'Filip',
       'Dushko', 'Filip', 'Filip', 'Filip', 'Dushko', 'Vesna', 'Vesna',
       'Dushko', 'Vesna', 'Filip', 'Filip', 'Dushko', 'Stefan', 'Dushko',
       'Filip', 'Filip', 'Filip', 'Vesna', 'Dushko', 'Vesna', 'Dushko',
       'Vesna', 'Vesna', 'Dushko', 'Dushko', 'Dushko', 'Vesna', 'Vesna',
       'Dushko', 'Vesna', 'Filip', 'Filip', 'Filip', 'Dushko', 'Filip',
       'Dushko', 'Filip', 'Filip', 'Vesna', 'Vesna', 'Vesna', 'Vesna',
       'Vesna', 'Dushko', 'Filip', 'Stefan', 'Filip', 'Filip', 'Vesna',
       'Vesna', 'Stefan', 'Dushko', 'Vesna', 'Filip', 'Filip', 'Dushko',
       'Dushko', 'Dushko', 'Dushko', 'Filip', 'Dushko', 'Vesna', 'Filip',
       'Dushko', 'Dushko', 'Stefan', 'Filip', 'Filip

In [55]:
# submission_df = pd.DataFrame({'Person': y_pred_classes})
submission_df = pd.DataFrame({'Person': y_pred_classes})
submission_df['id'] = np.arange(1, len(submission_df) + 1)
# submission_df['Person'] = y_test_pred
submission_df = submission_df[['id', 'Person']]
submission_df

,id,Person
0,1,Vesna
1,2,Filip
2,3,Vesna
3,4,Filip
4,5,Dushko
...,...,...
128,129,Vesna
129,130,Filip
130,131,Dushko
131,132,Dushko


In [56]:
submission_df.to_csv('my_pred.csv', index=False)

### predvidvanje so ensemble model

In [397]:
y_test_pred = voting_classifier.predict(X_test)
y_test_pred

array(['Stefan', 'Vesna', 'Vesna', 'Dushko', 'Filip', 'Vesna', 'Vesna',
       'Filip', 'Vesna', 'Stefan', 'Vesna', 'Filip', 'Stefan', 'Stefan',
       'Stefan', 'Filip', 'Filip', 'Stefan', 'Stefan', 'Vesna', 'Dushko',
       'Dushko', 'Filip', 'Filip', 'Vesna', 'Stefan', 'Stefan', 'Filip',
       'Stefan', 'Filip', 'Vesna', 'Filip', 'Vesna', 'Stefan', 'Stefan',
       'Stefan', 'Stefan', 'Filip', 'Vesna', 'Dushko', 'Filip', 'Dushko',
       'Vesna', 'Vesna', 'Filip', 'Stefan', 'Vesna', 'Stefan', 'Dushko',
       'Stefan', 'Stefan', 'Stefan', 'Filip', 'Dushko', 'Filip', 'Vesna',
       'Dushko', 'Vesna', 'Filip', 'Stefan', 'Filip', 'Dushko', 'Filip',
       'Filip', 'Filip', 'Filip', 'Stefan', 'Stefan', 'Stefan', 'Stefan',
       'Stefan', 'Vesna', 'Stefan', 'Stefan', 'Vesna', 'Vesna', 'Stefan',
       'Stefan', 'Vesna', 'Filip', 'Stefan', 'Vesna', 'Filip', 'Vesna',
       'Vesna', 'Stefan', 'Vesna', 'Vesna', 'Vesna', 'Stefan', 'Stefan',
       'Dushko', 'Vesna', 'Stefan', 'Vesna', 'Fi

In [398]:
# submission_df = pd.DataFrame({'Person': y_pred_classes})
submission_df = pd.DataFrame({'Person': y_test_pred})
submission_df['id'] = np.arange(1, len(submission_df) + 1)
# submission_df['Person'] = y_test_pred
submission_df = submission_df[['id', 'Person']]
submission_df

,id,Person
0,1,Stefan
1,2,Vesna
2,3,Vesna
3,4,Dushko
4,5,Filip
...,...,...
128,129,Vesna
129,130,Stefan
130,131,Stefan
131,132,Filip


In [399]:
indices = test_data.index[test_data['num_movements']==1].tolist()
for i in indices:
    submission_df.loc[i, 'Person'] = 'Dushko'
submission_df

,id,Person
0,1,Stefan
1,2,Vesna
2,3,Vesna
3,4,Dushko
4,5,Filip
...,...,...
128,129,Vesna
129,130,Stefan
130,131,Stefan
131,132,Filip


In [400]:
submission_df[submission_df['Person']=='Dushko']

,id,Person
3,4,Dushko
15,16,Dushko
20,21,Dushko
21,22,Dushko
31,32,Dushko
39,40,Dushko
41,42,Dushko
44,45,Dushko
48,49,Dushko
53,54,Dushko


In [401]:
submission_df.to_csv('my_pred_ensemble.csv', index=False)

### predvidvanje so rf model

In [434]:
y_test_pred = rf_model.predict(X_test)
# y_test_pred = rf_model_combined.predict(X_test)
y_test_pred

array(['Stefan', 'Vesna', 'Vesna', 'Filip', 'Stefan', 'Vesna', 'Vesna',
       'Filip', 'Vesna', 'Stefan', 'Vesna', 'Filip', 'Stefan', 'Stefan',
       'Stefan', 'Filip', 'Filip', 'Stefan', 'Stefan', 'Vesna', 'Stefan',
       'Dushko', 'Filip', 'Filip', 'Vesna', 'Stefan', 'Stefan', 'Dushko',
       'Stefan', 'Filip', 'Vesna', 'Filip', 'Vesna', 'Stefan', 'Stefan',
       'Stefan', 'Stefan', 'Filip', 'Vesna', 'Dushko', 'Filip', 'Dushko',
       'Filip', 'Vesna', 'Filip', 'Stefan', 'Vesna', 'Stefan', 'Dushko',
       'Stefan', 'Stefan', 'Stefan', 'Filip', 'Dushko', 'Filip', 'Vesna',
       'Dushko', 'Vesna', 'Filip', 'Stefan', 'Filip', 'Stefan', 'Filip',
       'Filip', 'Filip', 'Filip', 'Stefan', 'Stefan', 'Stefan', 'Stefan',
       'Stefan', 'Vesna', 'Stefan', 'Stefan', 'Vesna', 'Vesna', 'Stefan',
       'Stefan', 'Vesna', 'Filip', 'Stefan', 'Vesna', 'Filip', 'Vesna',
       'Vesna', 'Stefan', 'Stefan', 'Vesna', 'Vesna', 'Stefan', 'Stefan',
       'Stefan', 'Vesna', 'Stefan', 'Vesna', '

In [442]:
# submission_df = pd.DataFrame({'Person': y_pred_classes})
submission_df = pd.DataFrame({'Person': y_test_pred})
submission_df['id'] = np.arange(1, len(submission_df) + 1)
# submission_df['Person'] = y_test_pred
submission_df = submission_df[['id', 'Person']]
submission_df

,id,Person
0,1,Stefan
1,2,Vesna
2,3,Vesna
3,4,Filip
4,5,Stefan
...,...,...
128,129,Vesna
129,130,Stefan
130,131,Stefan
131,132,Filip


In [443]:
indices = test_data.index[test_data['num_movements']==2].tolist()
indices

[0,
 9,
 12,
 14,
 17,
 18,
 25,
 26,
 28,
 33,
 34,
 36,
 45,
 47,
 49,
 50,
 51,
 59,
 61,
 66,
 68,
 69,
 70,
 76,
 77,
 80,
 85,
 89,
 90,
 91,
 100,
 106,
 112,
 120,
 123,
 124,
 125,
 129,
 130]

In [444]:
for i in indices:
    submission_df.loc[i, 'Person'] = 'Dushko'
submission_df

,id,Person
0,1,Dushko
1,2,Vesna
2,3,Vesna
3,4,Filip
4,5,Stefan
...,...,...
128,129,Vesna
129,130,Dushko
130,131,Dushko
131,132,Filip


In [437]:
submission_df[submission_df['Person']=='Stefan']

,id,Person
0,1,Stefan
4,5,Stefan
9,10,Stefan
12,13,Stefan
13,14,Stefan
14,15,Stefan
17,18,Stefan
18,19,Stefan
20,21,Stefan
25,26,Stefan


In [445]:
submission_df.to_csv('my_pred.csv', index=False)

#### rf_combined (pomala tocnost na kaggle)

In [274]:
X_test = scaler_combined.transform(X_test)
X_test

array([[ 1.12655506e+00,  3.09122197e+00,  1.47304866e+00, ...,
        -4.06727488e-01, -2.33073135e-01, -5.28464465e-01],
       [ 6.06606570e-01,  3.73651876e-04,  2.60363697e-01, ...,
        -4.06727488e-01, -2.33073135e-01, -5.28464465e-01],
       [ 1.64650355e+00, -3.85982388e-01,  1.64628937e+00, ...,
         1.61761053e+00,  6.92061689e-01,  4.92904239e-01],
       ...,
       [-8.66580814e-02,  1.93215385e+00,  4.33604406e-01, ...,
        -9.12811992e-01, -6.95640547e-01, -1.03914882e+00],
       [-6.06606570e-01,  3.86729692e-01, -9.52321269e-01, ...,
        -4.06727488e-01, -2.33073135e-01, -5.28464465e-01],
       [ 1.29987122e+00,  3.73651876e-04,  7.80085825e-01, ...,
        -9.12811992e-01, -6.95640547e-01, -1.03914882e+00]])

In [275]:
y_test_pred = rf_model_combined.predict(X_test)
y_test_pred

array(['Vesna', 'Filip', 'Filip', 'Filip', 'Filip', 'Vesna', 'Filip',
       'Filip', 'Filip', 'Filip', 'Filip', 'Filip', 'Filip', 'Vesna',
       'Filip', 'Dushko', 'Filip', 'Filip', 'Dushko', 'Vesna', 'Filip',
       'Filip', 'Filip', 'Filip', 'Filip', 'Vesna', 'Dushko', 'Filip',
       'Dushko', 'Filip', 'Filip', 'Filip', 'Filip', 'Filip', 'Filip',
       'Dushko', 'Dushko', 'Filip', 'Filip', 'Filip', 'Filip', 'Filip',
       'Vesna', 'Vesna', 'Filip', 'Vesna', 'Filip', 'Dushko', 'Filip',
       'Dushko', 'Vesna', 'Dushko', 'Filip', 'Dushko', 'Filip', 'Vesna',
       'Filip', 'Filip', 'Filip', 'Filip', 'Filip', 'Filip', 'Filip',
       'Filip', 'Filip', 'Filip', 'Vesna', 'Filip', 'Dushko', 'Vesna',
       'Vesna', 'Filip', 'Filip', 'Dushko', 'Filip', 'Filip', 'Vesna',
       'Vesna', 'Filip', 'Filip', 'Filip', 'Filip', 'Filip', 'Filip',
       'Filip', 'Dushko', 'Filip', 'Vesna', 'Dushko', 'Dushko', 'Filip',
       'Dushko', 'Filip', 'Dushko', 'Filip', 'Filip', 'Filip', 'Filip',
   

In [276]:
# submission_df = pd.DataFrame({'Person': y_pred_classes})
submission_df = pd.DataFrame({'Person': y_test_pred})
submission_df['id'] = np.arange(1, len(submission_df) + 1)
# submission_df['Person'] = y_test_pred
submission_df = submission_df[['id', 'Person']]
submission_df

,id,Person
0,1,Vesna
1,2,Filip
2,3,Filip
3,4,Filip
4,5,Filip
...,...,...
128,129,Vesna
129,130,Filip
130,131,Dushko
131,132,Filip


In [277]:
submission_df.to_csv('my_pred.csv', index=False)